In [27]:
from keras.applications.resnet50 import ResNet50, preprocess_input

HEIGHT = 200
WIDTH = 200

base_model = ResNet50(weights='imagenet', 
                      include_top=False, 
                      input_shape=(HEIGHT, WIDTH,3))


In [18]:
from keras.layers import Dense, Activation, Flatten, Dropout
from keras.models import Sequential, Model

def build_finetune_model(base_model, dropout, fc_layers, num_classes):
    for layer in base_model.layers:
        layer.trainable = False

    x = base_model.output
    x = Flatten()(x)
    for fc in fc_layers:
        # New FC layer, random init
        x = Dense(fc, activation='relu')(x) 
        x = Dropout(dropout)(x)

    # New softmax layer
    predictions = Dense(num_classes, activation='softmax')(x) 
    
    finetune_model = Model(inputs=base_model.input, outputs=predictions)

    return finetune_model


In [51]:
class_list = ["0", "1"]
FC_LAYERS = [1024, 1024]
dropout = 0.5

finetune_model = build_finetune_model(base_model, 
                                      dropout=dropout, 
                                      fc_layers=FC_LAYERS, 
                                      num_classes=len(class_list))


In [23]:
import numpy as np
X_train=np.load("./X_train.npy")
Y_train=np.load("./Y_train.npy")
print(X_train.shape)

(229, 1, 200, 200)


In [29]:
X=X_train[:,0,:,:]


In [32]:
rgb_batch = np.repeat(X[..., np.newaxis], 3, -1)
print(rgb_batch.shape)

(229, 200, 200, 3)


In [45]:

from keras.optimizers import SGD, Adam
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
finetune_model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

In [46]:
finetune_model.fit(rgb_batch, Y_train, batch_size=32, epochs=10, verbose=1)

Epoch 1/10
229/229 [==============================] - 100s 435ms/step - loss: 9.0796 - acc: 0.4367
Epoch 2/10
229/229 [==============================] - 98s 428ms/step - loss: 9.0796 - acc: 0.4367
Epoch 3/10
229/229 [==============================] - 97s 425ms/step - loss: 9.0796 - acc: 0.4367
Epoch 4/10
229/229 [==============================] - 98s 427ms/step - loss: 9.0796 - acc: 0.4367
Epoch 5/10
229/229 [==============================] - 99s 434ms/step - loss: 9.0796 - acc: 0.4367
Epoch 6/10
229/229 [==============================] - 100s 438ms/step - loss: 9.0796 - acc: 0.4367
Epoch 7/10
229/229 [==============================] - 100s 437ms/step - loss: 9.0796 - acc: 0.4367
Epoch 8/10
229/229 [==============================] - 100s 436ms/step - loss: 9.0796 - acc: 0.4367
Epoch 9/10
229/229 [==============================] - 100s 437ms/step - loss: 9.0796 - acc: 0.4367
Epoch 10/10
229/229 [==============================] - 100s 437ms/step - loss: 9.0796 - acc: 0.4367


In [37]:
X_test=np.load("./X_test.npy")
Y_test=np.load("./Y_test.npy")
print(X_test.shape)

(100, 1, 200, 200)


In [41]:
Xtest=X_test[:,0,:,:]
Xtest_new = np.repeat(Xtest[..., np.newaxis], 3, -1)
print(Xtest_new.shape)


(100, 200, 200, 3)


In [47]:
score = finetune_model.evaluate(Xtest_new, Y_test, verbose=0)


In [48]:
print(score)


[7.897866864204406, 0.51]


In [52]:
adam = Adam(lr=0.00001)
finetune_model.compile(adam, loss='categorical_crossentropy', metrics=['accuracy'])


In [53]:

finetune_model.fit(rgb_batch, Y_train, batch_size=32, epochs=10, verbose=1)


Epoch 1/10
229/229 [==============================] - 104s 455ms/step - loss: 1.5548 - acc: 0.4585
Epoch 2/10
229/229 [==============================] - 104s 455ms/step - loss: 0.9909 - acc: 0.6638
Epoch 3/10
229/229 [==============================] - 105s 461ms/step - loss: 0.8701 - acc: 0.6987
Epoch 4/10
229/229 [==============================] - 103s 451ms/step - loss: 0.5102 - acc: 0.8166
Epoch 5/10
229/229 [==============================] - 104s 453ms/step - loss: 0.4087 - acc: 0.8472
Epoch 6/10
229/229 [==============================] - 104s 453ms/step - loss: 0.3365 - acc: 0.8821
Epoch 7/10
229/229 [==============================] - 104s 453ms/step - loss: 0.3242 - acc: 0.8908
Epoch 8/10
229/229 [==============================] - 104s 456ms/step - loss: 0.2929 - acc: 0.9083
Epoch 9/10
229/229 [==============================] - 104s 453ms/step - loss: 0.1475 - acc: 0.9345
Epoch 10/10
229/229 [==============================] - 103s 450ms/step - loss: 0.1470 - acc: 0.9345


In [54]:
score2 = finetune_model.evaluate(Xtest_new, Y_test, verbose=0)
print(score2)


[0.8435791015625, 0.49]


In [56]:
from keras.preprocessing.image import ImageDataGenerator

aug = ImageDataGenerator(rotation_range=20, zoom_range=0.15, width_shift_range=0.2, height_shift_range=0.2, shear_range=0.15, horizontal_flip=True, fill_mode="nearest")

In [61]:
EPOCHS = 10
BS = 32
H = finetune_model.fit_generator(aug.flow(rgb_batch[:200], Y_train[:200], batch_size=BS),
                                 validation_data=(rgb_batch[200:], Y_train[200:]), 
                                 steps_per_epoch=len(rgb_batch[:200]) // BS,epochs=EPOCHS)

Epoch 1/10
6/6 [==============================] - 91s 15s/step - loss: 1.3743 - acc: 0.6356 - val_loss: 1.4644 - val_acc: 0.0000e+00
Epoch 2/10
6/6 [==============================] - 96s 16s/step - loss: 0.9927 - acc: 0.6875 - val_loss: 1.2469 - val_acc: 0.0000e+00
Epoch 3/10
6/6 [==============================] - 84s 14s/step - loss: 1.0592 - acc: 0.6516 - val_loss: 1.3223 - val_acc: 0.0000e+00
Epoch 4/10
6/6 [==============================] - 74s 12s/step - loss: 1.0025 - acc: 0.5843 - val_loss: 1.4588 - val_acc: 0.0000e+00
Epoch 5/10
6/6 [==============================] - 95s 16s/step - loss: 0.9484 - acc: 0.6458 - val_loss: 1.3889 - val_acc: 0.0000e+00
Epoch 6/10
6/6 [==============================] - 83s 14s/step - loss: 1.0717 - acc: 0.6035 - val_loss: 1.2772 - val_acc: 0.0000e+00
Epoch 7/10
6/6 [==============================] - 82s 14s/step - loss: 1.1259 - acc: 0.6409 - val_loss: 1.2277 - val_acc: 0.0000e+00
Epoch 8/10
6/6 [==============================] - 82s 14s/step - loss